# NLLB-200 demo

This simple demo demonstrates how to load a distilled version of the [NLLB-200 model](https://github.com/facebookresearch/fairseq/tree/nllb), translate with it, and fine-tune it with a small additional corpus.

For more details, please refer to the GitHub repository linked above, the [Huggingface documentation](https://huggingface.co/docs/transformers/main/en/model_doc/nllb#nllb), and [our paper](https://research.facebook.com/publications/no-language-left-behind/).

In [ ]:
!nvidia-smi

Thu Apr 27 13:25:42 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers datasets evaluate numpy sacrebleu sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 46.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 468.7/468.7 kB 26.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.9/118.9 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 57.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 56.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

For this demo, we will be testing out translation from [Ligurian](https://en.wikipedia.org/wiki/Ligurian_language) into English.

In [6]:
SRC_LANG = "eng_Latn"
TGT_LANG = "hau_Latn"
TRAIN_BATCH_SIZE = 2

# Load data and create train/test/dev sets

In [9]:
#Paths

data_dir = ""

path_TMX_train = data_dir + "TMX_train.json"
path_TMX_dev = data_dir + "TMX_dev.json"
path_TMX_test = data_dir + "TMX_test.json"

path_FLORES_dev = data_dir + "eng_Latn.json"
path_FLORES_test = data_dir + "FLORES_test_hau-eng.json"

In [10]:
import json

# with open(path_FLORES_dev) as f:
#   data_FLORES_dev = json.loads(f.read())
with open(path_FLORES_test) as f:
  data_FLORES_test = json.loads(f.read())

In [11]:
# data_train = data_TMX_train
# data_dev = data_TMX_dev + data_FLORES_dev

# print("data_train", len(data_train))
# print("data_dev", len(data_dev))
print("data_test_FLORES", len(data_FLORES_test))
# print("data_test_TMX", len(data_TMX_test))

data_test_FLORES 1012


# Load model

In [12]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

model = AutoModelForSeq2SeqLM.from_pretrained("facebook/nllb-200-distilled-600M", use_cache=False)
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang=SRC_LANG, tgt_lang=TGT_LANG)

## 1. Inference

In [13]:
#Test inference
translator = pipeline("translation", model=model, tokenizer=tokenizer, device=0, src_lang=SRC_LANG, tgt_lang=TGT_LANG)

test_src_sentence = "Do you think this is right?"
print("Test inference before fine-tuning")
print("SRC:", test_src_sentence)
translator(test_src_sentence)

Test inference before fine-tuning
SRC: Do you think this is right?


/usr/local/lib/python3.9/dist-packages/transformers/generation/utils.py:1219: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation)
  warnings.warn(


[{'translation_text': 'Kana ganin wannan daidai ne?'}]

## 2. Fine-tuning

We create our own dataset for fine-tuning, using only a handful more examples:

In [ ]:
from datasets import Dataset, DatasetDict, load_dataset

src_key = "sentence_" + SRC_LANG
tgt_key = "sentence_" + TGT_LANG

# data = [
#     {src_key: "Emmo acciantou un chigheumao.", tgt_key: "We've planted a cucumber."},
#     {src_key: "Emmo acciantou un meigranâ.", tgt_key: "We've planted a pomegranate."},
#     {src_key: "Gh'é un chigheumao into frigo.", tgt_key: "There's a cucumber in the fridge."},
#     {src_key: "Te gusta o meigranâ?", tgt_key: "Do you like pomegranate?"},
#     {src_key: "Tutto insemme, o chigheumao e i faxolin vëgnan dexe euro.", tgt_key: "All together, the cucumber and the green beans are ten euros."},
#     {src_key: "O fruto do meigranâ o l'é ben ben doçe!", tgt_key: "The pomegranate fruit is very sweet!"},
#     {src_key: "O no te gusta o chigheumao?", tgt_key: "Don't you like the cucumber?"},
#     {src_key: "O chigheumao o no ne sa de ninte...", tgt_key: "Cucumbers don't taste of anything..."},
#     {src_key: "Ti gh'æ di chigheumai inte l'òrto?", tgt_key: "Do you have cucumbers in your vegetable garden?"},
#     {src_key: "Ò un chigheumao", tgt_key: "I have a cucumber"},
#     {src_key: "Ò un meigranâ", tgt_key: "I have a pomegranate"},
#     {src_key: "O mei e o chigheumao en di fruti.", tgt_key: "The apple and the cucumber are fruits."},
#     {src_key: "Mangemmo un chigheumao", tgt_key: "We eat a cucumber"},
#     {src_key: "Mangemmo un meigranâ", tgt_key: "We eat a pomegranate"},
# ]
data_finetune = Dataset.from_list(data_train)
# NB: We limit the size of the validation set so that it will run on this free
# instance of Colab. In practice you'd want to use the whole set.
# data_validate = load_dataset("facebook/flores", "lij_Latn-eng_Latn")["dev"].select(range(5))
data_validate = Dataset.from_list(data_dev)

We then prepare it by tokenising it:

In [ ]:
def tokenize_fn(examples):
  return tokenizer(examples[src_key], text_target=examples[tgt_key], padding="max_length", truncation=True)

tokenized_finetune = data_finetune.map(tokenize_fn, batched=True)
tokenized_validate = data_validate.map(tokenize_fn, batched=True)

Map:   0%|          | 0/73155 [00:00<?, ? examples/s]

Map:   0%|          | 0/1997 [00:00<?, ? examples/s]

We fine-tune the model with it:

In [ ]:
from transformers import TrainingArguments, Trainer, logging

# NB: We work with small batch sizes and checkpointing due to the limitations of
# this free instance of Colab. In practice you'd want to use settings closer to
# what we use in the paper.
training_args = TrainingArguments(
    output_dir="tmp",
    num_train_epochs=40,
    per_device_train_batch_size=TRAIN_BATCH_SIZE,
    per_device_eval_batch_size=TRAIN_BATCH_SIZE,
    gradient_accumulation_steps=4,
    eval_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True,
    fp16_full_eval=True,
    evaluation_strategy="epoch"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_finetune,
    eval_dataset=tokenized_validate,
)
trainer.train()

Using cuda_amp half precision backend
The following columns in the training set don't have a corresponding argument in `M2M100ForConditionalGeneration.forward` and have been ignored: sentence_knc_Latn, sentence_eng_Latn. If sentence_knc_Latn, sentence_eng_Latn are not expected by `M2M100ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 73155
  Num Epochs = 40
  Instantaneous batch size per device = 1
  Total train batch size (w. parallel, distributed & accumulation) = 4
  Gradient Accumulation steps = 4
  Total optimization steps = 731520
  Number of trainable parameters = 615073792


Epoch,Training Loss,Validation Loss


OutOfMemoryError: ignored

Let's try translating again, to see if things have improved:

In [ ]:
print("Test inference after fine-tuning")
print("SRC:", test_src_sentence)
translator(test_src_sentence)

[{'translation_text': 'A salad with tomatoes, pomegranate and cucumber cut into slices.'}]

In [ ]:
model.save_pretrained("my-fine-tuned-model")

Configuration saved in my-fine-tuned-model.pt/config.json
Configuration saved in my-fine-tuned-model.pt/generation_config.json
Model weights saved in my-fine-tuned-model.pt/pytorch_model.bin


# Test on test sets

## FLORES

In [14]:
from tqdm.notebook import tqdm
import sacrebleu

In [16]:
src_key = "sentence_" + SRC_LANG
tgt_key = "sentence_" + TGT_LANG

#Translate FLORES test set 
print("======Testing on FLORES======")
src_FLORES_test_eng = []
tgt_FLORES_test_hau = []
inf_FLORES_test_eng_hau = []
for sent in tqdm(data_FLORES_test):
  src = sent[src_key]
  tgt = sent[tgt_key]
  inf = translator(src)[0]['translation_text']

  src_FLORES_test_eng.append(src)
  tgt_FLORES_test_hau.append(tgt)
  inf_FLORES_test_eng_hau.append(inf)

======Testing on FLORES======


  0%|          | 0/1012 [00:00<?, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [19]:
#Save inference results to file
with open("inference_FLORES_eng_hau.txt", 'w', encoding='utf-8') as f:
  for l in inf_FLORES_test_eng_hau:
    f.write(l+"\n")

In [21]:
print("First sample")
print("SRC:", src_FLORES_test_eng[0])
print("TGT:", tgt_FLORES_test_hau[0])
print("INF:", inf_FLORES_test_eng_hau[0])

First sample
SRC: "We now have 4-month-old mice that are non-diabetic that used to be diabetic," he added.
TGT: “Yanzu muna bera mai watanni 4 wadanda ba su da ciwon sukari ba wadanda ke da suke fama da ciwon sukari,” in ji shi.
INF: Ya kara da cewa: "Yanzu muna da berayen da ba su da ciwon sukari kuma suka yi shekara huɗu da haihuwa.


In [22]:
# Calculate BLEU
bleu = sacrebleu.corpus_bleu(inf_FLORES_test_eng_hau, [tgt_FLORES_test_hau], tokenize='flores200')
print("BLEU:", round(bleu.score, 2))

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(inf_FLORES_test_eng_hau, [tgt_FLORES_test_hau])
print("CHRF:", round(chrf.score, 2))

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(inf_FLORES_test_eng_hau, [tgt_FLORES_test_hau])
print("TER:", round(ter.score, 2))   

BLEU: 24.32
CHRF: 49.83
TER: 68.0


## Test on TMX

In [ ]:
#Translate TMX test set 
print("======Testing on TMX======")
src_TMX_test_eng = []
tgt_TMX_test_knc = []
inf_TMX_test_eng_knc = []
for sent in tqdm(data_TMX_test):
  src = sent[src_key]
  tgt = sent[tgt_key]
  inf = translator(src)[0]['translation_text']

  src_TMX_test_eng.append(src)
  tgt_TMX_test_knc.append(tgt)
  inf_TMX_test_eng_knc.append(inf)

======Testing on TMX======


  0%|          | 0/1000 [00:00<?, ?it/s]

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1045: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Generate config GenerationConfig {
  "bos_token_id": 0,
  "decoder_start_token_id": 2,
  "eos_token_id": 2,
  "max_length": 200,
  "pad_token_id": 1,
  "transformers_version": "4.26.1",
  "use_cache": false
}



In [ ]:
#Save inference results to file
with open("inference_TMX_eng_knc.txt", 'w') as f:
  for l in inf_TMX_test_eng_knc:
    f.write(l+"\n")

In [ ]:
print("First sample")
print("SRC:", src_TMX_test_eng[0])
print("TGT:", tgt_TMX_test_knc[0])
print("INF:", inf_TMX_test_eng_knc[0])

First sample
SRC: Do you think this is right?
TGT: Akaidǝ kalkallo ruwinna?
INF: Adǝ shima zauro kәla ro waljin?


In [ ]:
# Calculate BLEU
bleu = sacrebleu.corpus_bleu(inf_TMX_test_eng_knc, [tgt_TMX_test_knc], tokenize='flores200')
print("BLEU:", round(bleu.score, 2))

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(inf_TMX_test_eng_knc, [tgt_TMX_test_knc])
print("CHRF:", round(chrf.score, 2))

# Calculate TER
metric = sacrebleu.metrics.TER()
ter = metric.corpus_score(inf_TMX_test_eng_knc, [tgt_TMX_test_knc])
print("TER:", round(ter.score, 2))

BLEU: 4.03
CHRF: 15.45
TER: 200.0
